# <font color=#B2D732> <span style="background-color: #4424D6"> Spinal cord microstructural preprocessings </font>
<hr style="border:1px solid black">  

*Project: 2024_brsc_aging_project*  
*Paper: in prep*  
**@ author:** 
> Caroline Landelle, caroline.landelle@mcgill.ca // landelle.caroline@gmail.com   
> June 2024 (last update: 4 June 2024)

**Description:** 
> This notebook provides code for microstructural MRI data of spinal cord acquisition.
> The second line of the section "imports & configuration" of this notebook should be modify with the good path 


**Associated files:**
> - ../config/02_brsc_preprocess_microstructural*.json ; This file contains the directories and should be modified first
> - ../config/participants_brsc_*.tsv ; this file contains the information about the participants and should be updated after each new acquisition
> - ../brsc_preprocess.py ; Nothing should be changed in this file
> - ../BrSc_utils.py; Nothing should be changed in this file
> - You need to copy the version sct6.1 of the template and atlas that include spinal segmental levels + WM tracts and put the path in the config file "PAM50_sct_dir":".../template_sct6.1",
 

**Steps:**
> **I. T2ws image**  
> I.1 Selct files
> I.2 Segmentation 
> I.2 Registration into PAM50 space
 

<hr style="border:1px solid black">



## <font color=#B2D732> <span style="background-color: #4424D6">  Imports & configuration </font >

In [52]:
import sys, json, glob, os, shutil
code_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/2025_brsc_aging_project/"
os.chdir(code_dir)
sys.path.append(code_dir + "/code/") # Change this line according to your directory

from brsc_preprocess import Preprocess_BrSc, Preprocess_Sc, Preprocess_Br
import brsc_utils as utils
#
%load_ext autoreload
%autoreload 2
%matplotlib inline
    
# Load config file ------------------------------------------------------------
# The config file should be first review and all path should be modified (lines: )
with open(code_dir + '/config/preprocessing/03_sc_preprocess_microstructural.json') as config_file: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_file) # load config file should be open first and the path inside modified
    
preprocess_BrSc=Preprocess_BrSc(config) # initialize the function
preprocess_Sc=Preprocess_Sc(config) # initialize the function
preprocess_Br=Preprocess_Br(config) # initialize the function


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The config files should be manually modified first
All the raw data should be store in BIDS format
 


<hr style="border:1px solid black">

## <font color=#B2D732> <span style="background-color: #4424D6"> I. T2w preprocessings </font>

#### I.1 Select file and copy if necessary

In [20]:
T2w_raw_files=[];T2w_files=[]; warpT1w_PAM50_files=[];warpPAM50_T1w_files=[]
for ID_nb,ID in enumerate(config["participants_IDs"]):
    tag_anat=""
    preprocess_dir=config["main_dir"]+ config["preprocess_dir"]["bmpd_dir"] if ID[0]=="P" else config["main_dir"] +config["preprocess_dir"]["main_dir"]
    raw_dir= config["main_dir"]+ config["bmpd_raw_dir"] if ID[0]=="P" else config["main_dir"]+ config["raw_dir"]
    IDbis=ID#

    if ID in config['files_specificities']["T2w"]:
        tag_anat=config['files_specificities']["T2w"][ID] # if you provided filename specifity it will be take into account 

    if ID in config["double_IDs"]:
        IDbis=config["double_IDs"][ID]

    T2w_raw_files.append(glob.glob(raw_dir + "/sub-" + ID+ "/"+ config["design_exp"]["ses_names"][0]+ "/anat/sub-"+IDbis+tag_anat+"_T2w.nii*")[0])
    if ID in config["double_IDs"]:
        T2w_files.append(preprocess_dir + "sub-" + ID + "/anat/sub-" + ID + os.path.basename(T2w_raw_files[ID_nb]).split(ID[1:4])[1])
    else:
        T2w_files.append(preprocess_dir + "sub-" + ID + "/anat/" +os.path.basename(T2w_raw_files[ID_nb]))

    if not os.path.exists(T2w_files[ID_nb]):
        shutil.copy(T2w_raw_files[ID_nb], T2w_files[ID_nb])

    warpT1w_PAM50_files.append(glob.glob(preprocess_dir + "sub-" + ID + "/anat/"+ config["preprocess_dir"]["T1w_sc_coreg"] + "*from-T1w_to-PAM50*")[0])
    warpPAM50_T1w_files.append(glob.glob(preprocess_dir + "sub-" + ID + "/anat/"+ config["preprocess_dir"]["T1w_sc_coreg"] + "*from-PAM50_to-T1w*")[0])

    

In [31]:
T2w_files=[]
for ID_nb,ID in enumerate(config["participants_IDs"]):
    T2w_files.append(preprocess_dir + "sub-" + ID + "/anat/sub-" + ID + os.path.basename(T2w_raw_files[ID_nb]).split(ID[1:4])[1])
T2w_files    

['/cerebro/cerebro1/dataset/stratals/derivatives/preprocessing/sub-A006_test/anat/sub-A006_test_T2w.nii.gz']

### I.2 Segmentation
#### a segment the spinal cord (WM+GM)
<span style="background-color: #FFFACD"> <font color=#efb017> **/!\**  </font>  In this dataset the segmentation mask were all corrected manually by overwriting the file: **/func///4_Segmentation//spinalcord/*seg.nii.gz  </span> 

In [32]:
seg_gm_T2w_files=[];seg_cord_T2w_files=[] # will contain the output filename for all participants.
seg_wm_T2w_files=[];seg_csf_T2w_files=[]
for ID_nb, ID in enumerate(config["participants_IDs"]):
    seg_cord_T2w_files.append(preprocess_Sc.segmentation(ID=ID,i_img=T2w_files[ID_nb], ses_name=config["design_exp"]["ses_names"][0],img_type="T2w",contrast_anat="t2s",tag="_cord_seg.nii.gz",redo=False,verbose=True))
    seg_gm_T2w_files.append(preprocess_Sc.segmentation(ID=ID,i_img=T2w_files[ID_nb], ses_name=config["design_exp"]["ses_names"][0],img_type="T2w",contrast_anat="t2s",tissue="gm",tag="_gm_seg.nii.gz",redo=False,verbose=True))
    seg_wm_T2w_files.append(preprocess_Sc.segmentation(ID=ID,i_img=seg_cord_T2w_files[ID_nb], i_gm_img=seg_gm_T2w_files[ID_nb],ses_name=config["design_exp"]["ses_names"][0],img_type="T2w",contrast_anat="t2s",tissue="wm",tag="_wm_seg.nii.gz",redo=False,verbose=True))
    seg_csf_T2w_files.append(preprocess_Sc.segmentation(ID=ID,i_img=T2w_files[ID_nb], ses_name=config["design_exp"]["ses_names"][0],img_type="T2w",contrast_anat="t2s",tissue="csf",tag="_propseg.nii.gz",redo=False,verbose=True))
    
 

/cerebro/cerebro1/dataset/stratals/derivatives/preprocessing/sub-A006_test//anat//T2w_aging_project/segmentation/
>>>>> Segmentation file already exists for the T2wimage of the sub-A006_test 
fsleyes /cerebro/cerebro1/dataset/stratals/derivatives/preprocessing/sub-A006_test//anat//T2w_aging_project/segmentation/sub-A006_test_T2w_cord_seg.nii.gz
 
/cerebro/cerebro1/dataset/stratals/derivatives/preprocessing/sub-A006_test//anat//T2w_aging_project/segmentation/
>>>>> Segmentation file already exists for the T2wimage of the sub-A006_test 
fsleyes /cerebro/cerebro1/dataset/stratals/derivatives/preprocessing/sub-A006_test//anat//T2w_aging_project/segmentation/sub-A006_test_T2w_gm_seg.nii.gz
 
/cerebro/cerebro1/dataset/stratals/derivatives/preprocessing/sub-A006_test//anat//T2w_aging_project/segmentation/
>>>>> Segmentation file already exists for the T2wimage of the sub-A006_test 
fsleyes /cerebro/cerebro1/dataset/stratals/derivatives/preprocessing/sub-A006_test//anat//T2w_aging_project/segm

### VI.2 Coregistration to PAM50 template
#### a Coregistration between T2w image and PAM50
https://spinalcordtoolbox.com/user_section/tutorials/gray-matter-segmentation/improving-registration-with-gm-seg/gm-informed-t2s-template-registration.html
T2* scans are typically acquired axially with thick slices, it is much more difficult to acquire the vertebral labels needed for the vertebral matching step.
Thus we use  sct_register_multimodal with initialization (warping field) between T1w and PAM50 were we were abled to extract the vertebral labels.

In [39]:
T2w2PAM50_dirs=[];o_warp_imgs=[];o_warpinv_imgs=[] # will contain the output filename for all participants.
template_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/templates/PAM50/template/"
    
for ID_nb, ID in enumerate(config["participants_IDs"]):
    
    # coregistration with PAM50
    T2w2PAM50=preprocess_Sc.coreg_img2PAM50(ID=ID,i_img=T2w_files[ID_nb],
                                            i_seg=seg_wm_T2w_files[ID_nb],
                                            PAM50_cord=config["tools_dir"]["main_codes"] +config["PAM50_wm"],
                                            PAM50_t2=config["tools_dir"]["main_codes"] +config["PAM50_t2"],
                                            ses_name=config["design_exp"]["ses_names"][0],
                                            o_folder=os.path.dirname(T2w_files[ID_nb]) + config["preprocess_dir"]["T2w_sc_coreg"],
                                            initwarp=warpT1w_PAM50_files[ID_nb],initwarpinv=warpPAM50_T1w_files[ID_nb],
                                            img_type="t2s",
                                            redo=False,verbose=True)
    
    T2w2PAM50_dirs.append(T2w2PAM50[0])
    o_warp_imgs.append(T2w2PAM50[1])
    o_warpinv_imgs.append(T2w2PAM50[2])

    # wrap PAM50 files into T2w space, this step requiered the v6.1 of the template to include the segmental levels (config["PAM50_sct_dir"])
    
    if not os.path.exists(T2w2PAM50_dirs[ID_nb] + "/template/PAM50_cord.nii.gz"):
        string1="sct_warp_template -d "+T2w_files[ID_nb]+" -w " + o_warpinv_imgs[ID_nb] + " -a 1 -ofolder "+T2w2PAM50_dirs[ID_nb]+" -t " +config["PAM50_sct_dir"]
        os.system(string1)

        # add home made atlas (spinal levels from icaps)
        function="/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.6.0/bin/isct_antsApplyTransforms"
        for atlas in ["_" , "_gm_","_wm_"]:
                string=function + " -d 3 -i " +config["PAM50_sct_dir"] + "/template/PAM50_Frostell"+atlas+"28Parcels_7Networks.nii.gz -o " +T2w2PAM50_dirs[ID_nb] +"/template/PAM50_Frostell"+atlas+"28Parcels_7Networks.nii.gz -t " + o_warpinv_imgs[ID_nb] +" -r " +T2w_files[ID_nb] + " -n NearestNeighbor"
                os.system(string)
    # Apply transformation for gm and wm masks into PAM50
    for i_img in[seg_gm_T2w_files,seg_wm_T2w_files,seg_cord_T2w_files]:
        preprocess_Sc.apply_warp(i_img=i_img,ID=config["participants_IDs"],
                                o_folder=T2w2PAM50_dirs,
                                 warping_field=o_warp_imgs,
                                 threshold=0.15,
                                 ses_name=config["design_exp"]["ses_names"][0],tag='_inPAM50',
                                 redo=False,n_jobs=8)
        


>>>>> Registration between func image and PAM50 already exists for sub-A006_test
 


<hr style="border:1px solid black">

## <font color=#B2D732> <span style="background-color: #4424D6"> II. Magnetisation transfert preprocessings </font>

### II.1 Select file and copy if necessary

In [43]:
MTon_raw_files=[];MTon_files=[];MToff_raw_files=[];MToff_files=[]
for ID_nb,ID in enumerate(config["participants_IDs"]):
    tag_anat=""
    preprocess_dir=config["main_dir"]+ config["preprocess_dir"]["bmpd_dir"] if ID[0]=="P" else config["main_dir"] +config["preprocess_dir"]["main_dir"]
    raw_dir= config["main_dir"]+ config["bmpd_raw_dir"] if ID[0]=="P" else config["main_dir"]+ config["raw_dir"]
    IDbis=ID
    if ID in config['files_specificities']["MT"]:
        tag_anat=config['files_specificities']["MT"][ID] # if you provided filename specifity it will be take into account 

    if ID in config["double_IDs"]:
        IDbis=config["double_IDs"][ID]

    MTon_raw_files.append(glob.glob(raw_dir + "/sub-" + ID+ "/"+ config["design_exp"]["ses_names"][0]+ "/anat/sub-"+IDbis+ "*MTonMTS*"+tag_anat +"*.nii*")[0])
    MToff_raw_files.append(glob.glob(raw_dir + "/sub-" + ID+ "/"+ config["design_exp"]["ses_names"][0]+ "/anat/sub-"+IDbis+ "*MToffMTS*"+tag_anat +"*.nii*")[0])
    
    if ID in config["double_IDs"]:
        MTon_files.append(preprocess_dir + "sub-" + ID + "/anat/sub-" + ID + os.path.basename(MTon_raw_files[ID_nb]).split(ID[1:4])[1])
        MToff_files.append(preprocess_dir + "sub-" + ID + "/anat/sub-" + ID + os.path.basename(MToff_raw_files[ID_nb]).split(ID[1:4])[1])
    else:
        MTon_files.append(preprocess_dir + "sub-" + ID + "/anat/" +os.path.basename(MTon_raw_files[ID_nb]))
        MToff_files.append(preprocess_dir + "sub-" + ID + "/anat/" +os.path.basename(MToff_raw_files[ID_nb]))

    if not os.path.exists(MTon_files[ID_nb]):
        shutil.copy(MTon_raw_files[ID_nb], MTon_files[ID_nb])
        shutil.copy(MToff_raw_files[ID_nb], MToff_files[ID_nb])
    

### II.1 Segmentation and mask

In [45]:
seg_MTon_files=[];mask_MTon_files=[]

for ID_nb in range(0,len(config["participants_IDs"])):
    
   
    ID=config["participants_IDs"][ID_nb]
    # segmentation
    seg_MTon_files.append(preprocess_Sc.segmentation(ID=ID,
                                                     i_img=MTon_files[ID_nb], 
                                                     ses_name=config["design_exp"]["ses_names"][0],
                                                     img_type="MT",
                                                     contrast_anat="t2",
                                                     tissue="cord",
                                                     tag="_cord_seg.nii.gz",
                                                     redo=False))

    #mask
    mask_MTon_files.append(seg_MTon_files[ID_nb].split('seg.nii.gz')[0] + 'mask.nii.gz')
    if not os.path.exists(mask_MTon_files[ID_nb]):
        string="sct_create_mask -i "+MTon_files[ID_nb] +" -p centerline,"+seg_MTon_files[ID_nb]+" -size 35mm -o " + mask_MTon_files[ID_nb]
        os.system(string)
 

/cerebro/cerebro1/dataset/stratals/derivatives/preprocessing/sub-A006_test//anat//MT_aging_project/segmentation/
>>>>> Segmentation file already exists for the MTimage of the sub-A006_test 
fsleyes /cerebro/cerebro1/dataset/stratals/derivatives/preprocessing/sub-A006_test//anat//MT_aging_project/segmentation/sub-A006_test_acq-MTonMTS_T1w_cord_seg.nii.gz
 


### II.4 Register MToff to MTon data

In [47]:
MToff_reg=[]
params = f"step=1,type=im,algo=slicereg,metric=CC"
for ID_nb in range(0,len(config["participants_IDs"])):
    ID=config["participants_IDs"][ID_nb]
    preprocess_dir=config["main_dir"]+ config["preprocess_dir"]["bmpd_dir"] if ID[0]=="P" else config["main_dir"] +config["preprocess_dir"]["main_dir"]

    o_folder=preprocess_dir + "sub-" + ID + "/anat/" + config["preprocess_dir"]["MT_sc_coreg"]
    MToff_reg.append(o_folder + os.path.basename(MToff_files[ID_nb]).split(".")[0] + "_reg.nii.gz")
    if not os.path.exists(o_folder):
        os.mkdir(o_folder)
    if not os.path.exists(MToff_reg[ID_nb]):
        string="sct_register_multimodal -i "+MToff_files[ID_nb] + " -d " +MTon_files[ID_nb] + " -dseg " + seg_MTon_files[ID_nb] + " -m " + mask_MTon_files[ID_nb] + " -param " +params + " -x spline -ofolder " + o_folder
        os.system(string)
       

### II.5 Compute mtr

In [49]:
mtr_file=[]
for ID_nb in range(0,len(config["participants_IDs"])):
    ID=config["participants_IDs"][ID_nb]
    preprocess_dir=config["main_dir"]+ config["preprocess_dir"]["bmpd_dir"] if ID[0]=="P" else config["main_dir"] +config["preprocess_dir"]["main_dir"]
    o_folder=preprocess_dir + "sub-" + ID + "/anat/" + config["preprocess_dir"]["MT_sc_mtr"]
    mtr_file.append(o_folder + "/sub-" + ID + "_MTR.nii.gz")
    if not os.path.exists(o_folder):
        os.mkdir(o_folder)
    if not os.path.exists(mtr_file[ID_nb]):
        string="sct_compute_mtr -mt0 " + MToff_reg[ID_nb] + ' -mt1 ' + MTon_files[ID_nb] + " -o " + mtr_file[ID_nb]
        os.system(string)   

### II.6 Register to PAM50 template

In [55]:
MTR2PAM50_dirs=[];o_warp_imgs=[];o_warpinv_imgs=[] # will contain the output filename for all participants.
for ID_nb, ID in enumerate(config["participants_IDs"]):
    ID=config["participants_IDs"][ID_nb]
    params = 'step=1,type=seg,algo=centermass,metric=MeanSquares:step=2,algo=bsplinesyn,type=seg,slicewise=1,iter=5'
    
    # coregistration with PAM50
    MTR2PAM50=preprocess_Sc.coreg_img2PAM50(ID=ID,i_img=mtr_file[ID_nb],
                                            i_seg=seg_MTon_files[ID_nb],
                                            PAM50_cord=config["tools_dir"]["main_codes"] +config["PAM50_cord"],
                                            PAM50_t2=config["tools_dir"]["main_codes"] + config["PAM50_t2"],#coreg_type="centermass",
                                            ses_name=config["design_exp"]["ses_names"][0],
                                            o_folder=os.path.dirname(MTon_files[ID_nb]) + config["preprocess_dir"]["MT_sc_coreg"],
                                            initwarp=warpT1w_PAM50_files[ID_nb],initwarpinv=warpPAM50_T1w_files[ID_nb],
                                            img_type="mtr",
                                            redo=False,verbose=True)
    
    MTR2PAM50_dirs.append(MTR2PAM50[0])
    o_warp_imgs.append(MTR2PAM50[1])
    o_warpinv_imgs.append(MTR2PAM50[2])

    if not os.path.exists(MTR2PAM50_dirs[ID_nb] + "/template/PAM50_cord.nii.gz"):
        string="sct_warp_template -d "+mtr_file[ID_nb]+" -w " + o_warpinv_imgs[ID_nb] + " -a 1 -ofolder "+MTR2PAM50_dirs[ID_nb]+" -t " +config["PAM50_sct_dir"]
        os.system(string)
    
        # add home made atlas (spinal levels from icaps)) 
        function="/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.6.0/bin/isct_antsApplyTransforms"
        for atlas in ["_" , "_gm_","_wm_"]:
            string=function + " -d 3 -i " +config["PAM50_sct_dir"] + "/template/PAM50_Frostell"+atlas+"28Parcels_7Networks.nii.gz -o " +MTR2PAM50_dirs[ID_nb] +"/template/PAM50_Frostell"+atlas+"28Parcels_7Networks.nii.gz -t " + o_warpinv_imgs[ID_nb] +" -r " +mtr_file[ID_nb] + " -n NearestNeighbor"
            os.system(string)

        for wm_atlas in ["3tracts"]:
            string=function + " -d 3 -i " +config["PAM50_sct_dir"] + "/template/PAM50_wm_atlas_"+wm_atlas+".nii.gz -o " +MTR2PAM50_dirs[ID_nb] +"/template/PAM50_wm_atlas_"+wm_atlas+".nii.gz -t " + o_warpinv_imgs[ID_nb] +" -r " +mtr_file[ID_nb] + " -n NearestNeighbor"
            os.system(string)


>>>>> Registration between func image and PAM50 already exists for sub-A006_test
 
